In [1]:
import torch
from torchvision.models import vgg19
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
from torch import nn
from PIL import Image
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

In [2]:
model = vgg19(pretrained=True)
model.classifier[6] = nn.Linear(4096, 1)

In [3]:
compleete_dataset = pd.DataFrame()

In [4]:
cats = [os.path.join('../../dataset/training_set/training_set/cats', x) for x in os.listdir('../../dataset/training_set/training_set/cats')]
dogs = [os.path.join('../../dataset/training_set/training_set/dogs', x) for x in os.listdir('../../dataset/training_set/training_set/dogs')]

In [5]:
dogs_frame = pd.DataFrame(dogs, columns=['path'])
dogs_frame['label'] = 1

cats_frame = pd.DataFrame(cats, columns=['path'])
cats_frame['label'] = 0

result_dataset = dogs_frame.append(cats_frame).reset_index(drop=True)

C:\Users\yakub\AppData\Local\Temp\ipykernel_1308\2438378232.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataset = dogs_frame.append(cats_frame).reset_index(drop=True)


In [6]:
train_sample = result_dataset.sample(int(0.75*len(result_dataset)))
test_sample = result_dataset.drop(train_sample.index).reset_index(drop=True)
train_sample = train_sample.reset_index(drop=True)

In [7]:
test_imgs = test_sample.path.apply(lambda x: x.split('/')[1])

In [9]:
class MyDataset(Dataset):
    def __init__(self, dataset, train=True):
        self.dataset = dataset
        self.train = train
    def __len__(self):
        return(len(self.dataset))

    def __getitem__(self, item):
        sample = self.dataset.iloc[item]
        img_label = sample.label
        img_path = sample.path

        img = np.asarray(Image.open(img_path))

        augumentations = A.Sequential(
            [
                A.Resize(128, 128),
                A.HorizontalFlip(p=0.5) if self.train else A.HorizontalFlip(p=0),
                A.ColorJitter(p=0.3) if self.train else A.ColorJitter(p=0),
                A.Normalize(),
                ToTensorV2()
            ]
        )

        img = augumentations(image=img)['image']
        return img, img_label

In [10]:
loss_function = nn.BCEWithLogitsLoss()

In [11]:
train_dataset = MyDataset(train_sample, train=True)
test_datset = MyDataset(test_sample, train=False)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_datset, batch_size=16, shuffle=True, drop_last=False)

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = vgg19(pretrained=True)
model.classifier[6] = nn.Linear(4096, 1)
model = model.to(device)
num_epochs = 2

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)


In [13]:
train_losses = []
test_losses = []
for epoch in range(num_epochs):
    print('epoch ', epoch)
    model.train()

    train_loss = 0
    test_loss = 0

    for img, label in tqdm(train_dataloader):
        img = img.to(device)
        label = label.to(device).float()

        predictions = model(img).squeeze()
        loss = loss_function(predictions, label)

        model.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss/=len(train_dataloader)
    train_losses.append(train_loss)

    model.eval()
    for img, label in tqdm(test_dataloader):
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device).float()
            predictions = model(img).squeeze()
            loss = loss_function(predictions, label)
            test_loss += loss.item()

    test_loss /= len(test_dataloader)
    test_losses.append(test_loss)

    torch.cuda.empty_cache()

    print('train loss: ', train_loss)
    print('test loss: ', test_loss)


epoch  0


100%|██████████| 126/126 [00:27<00:00,  4.64it/s]


train loss:  0.10944142518968632
test loss:  0.06700779886402113
epoch  1


  6%|▋         | 24/375 [00:04<01:03,  5.56it/s]


KeyboardInterrupt: 

In [14]:
torch.save(model.state_dict(), 'classificator.pth')

In [15]:
all_predictions = []
all_reals = []
model.eval()
for img, label in tqdm(test_dataloader):
    with torch.no_grad():
        img = img.to(device)
        label = list(label.numpy())
        predictions = list(nn.Sigmoid()(model(img)).squeeze().cpu().detach().numpy())
        all_predictions += predictions
        all_reals += label


100%|██████████| 126/126 [00:07<00:00, 16.68it/s]


In [16]:
preds = (np.array(all_predictions)>0.5).astype(int)

In [17]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(all_reals, preds))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1018
           1       0.97      0.98      0.98       984

    accuracy                           0.98      2002
   macro avg       0.98      0.98      0.98      2002
weighted avg       0.98      0.98      0.98      2002



In [19]:
### Посмотрим на результат классификации на плохом качестве

In [23]:
cats = [os.path.join('cats_bad/', x) for x in os.listdir('cats_orig/')]
dogs = [os.path.join('dogs_bad/', x) for x in os.listdir('dogs_orig/')]
dogs_frame = pd.DataFrame(dogs, columns=['path'])
dogs_frame['label'] = 1

cats_frame = pd.DataFrame(cats, columns=['path'])
cats_frame['label'] = 0

result_dataset = dogs_frame.append(cats_frame).reset_index(drop=True)

val_dataset = MyDataset(result_dataset, train=False)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, drop_last=False)

C:\Users\yakub\AppData\Local\Temp\ipykernel_1308\2775868923.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataset = dogs_frame.append(cats_frame).reset_index(drop=True)


In [24]:
all_predictions = []
all_reals = []
model.eval()
for img, label in tqdm(val_dataloader):
    with torch.no_grad():
        img = img.to(device)
        label = list(label.numpy())
        predictions = list(nn.Sigmoid()(model(img)).squeeze().cpu().detach().numpy())
        all_predictions += predictions
        all_reals += label

preds = (np.array(all_predictions)>0.5).astype(int)
print(classification_report(all_reals, preds))

100%|██████████| 64/64 [00:06<00:00, 10.27it/s]

              precision    recall  f1-score   support

           0       0.95      0.67      0.78      1011
           1       0.74      0.96      0.84      1012

    accuracy                           0.82      2023
   macro avg       0.85      0.82      0.81      2023
weighted avg       0.85      0.82      0.81      2023



In [25]:
### Посмотрим на результат классификации на восстановленном качестве

In [26]:
cats = [os.path.join('cats_refactored/', x) for x in os.listdir('cats_orig/')]
dogs = [os.path.join('dogs_refactored/', x) for x in os.listdir('dogs_orig/')]
dogs_frame = pd.DataFrame(dogs, columns=['path'])
dogs_frame['label'] = 1

cats_frame = pd.DataFrame(cats, columns=['path'])
cats_frame['label'] = 0

result_dataset = dogs_frame.append(cats_frame).reset_index(drop=True)

val_dataset = MyDataset(result_dataset, train=False)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, drop_last=False)

C:\Users\yakub\AppData\Local\Temp\ipykernel_1308\876080052.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataset = dogs_frame.append(cats_frame).reset_index(drop=True)


In [27]:
all_predictions = []
all_reals = []
model.eval()
for img, label in tqdm(val_dataloader):
    with torch.no_grad():
        img = img.to(device)
        label = list(label.numpy())
        predictions = list(nn.Sigmoid()(model(img)).squeeze().cpu().detach().numpy())
        all_predictions += predictions
        all_reals += label

preds = (np.array(all_predictions)>0.5).astype(int)
print(classification_report(all_reals, preds))

100%|██████████| 64/64 [00:06<00:00,  9.96it/s]

              precision    recall  f1-score   support

           0       0.91      0.94      0.92      1011
           1       0.94      0.90      0.92      1012

    accuracy                           0.92      2023
   macro avg       0.92      0.92      0.92      2023
weighted avg       0.92      0.92      0.92      2023

